# Compare MEL parameters, using PESQ MOS, for a range of MEL implementations.

- First test uses complex numbers and doesn't do any power spectrum compression.
- Second test uses the same code as the first test but does log power compression.
- Third test uses Griffin-Lim to estimate the missing phase info during reconstruction.

See conclusions at the end for recommended settings.

In [1]:
import os
tools_path = os.path.join(os.getcwd(), '..', 'Tools')
print("Tools:", tools_path)

pesq_path = os.path.join(tools_path, 'pesq.exe')
print('PESQ:', pesq_path)

#Add tools dir to path
import sys
sys.path.append(tools_path)

import AVPreprocess as avp
import WavMelConverter as wmc
import numpy as np
import librosa
from scipy.io import wavfile

#Paths - assumes notebook is running in a experiment directory.
base_dir = os.path.join(os.getcwd())

data_dir = os.path.join(base_dir, 'data')
print("Data dir:", data_dir)

input_dir = os.path.join(data_dir, 'raw')
base_output_dir = os.path.join(data_dir, 'results')
os.makedirs(base_output_dir, exist_ok=True)

Tools: D:\Code\Denoising\E0-Mel\..\Tools
PESQ: D:\Code\Denoising\E0-Mel\..\Tools\pesq.exe
Data dir: D:\Code\Denoising\E0-Mel\data


In [2]:
#Shared range of values to try

start_freq = 300.0
end_freq = 8000.0

#Values outside these ranges led to combinations that created empty bins, non-real values or internal errors because of mis-sized arrays.
sr_range = [16000, 32000, 48000]
fft_range = [1024, 2048]
step_range = [32, 64]
mel_bin_range = [80, 100, 120]

#test_file = os.path.join(input_dir, 'bush-16k.wav') #sample used in original WAV->MEL->WAV code
#output_file_base = "bush"
#Best MEL parameters and PESQ score: ((16000, 1024, 64, 120), 2.879)
#Also good: 32000 1024 64 120 = 2.824

test_file = os.path.join(input_dir, '17_en-US-Wavenet-A_0_14-2-C.wav') #one sec of aviation text to speech
output_file_base = "tts"


In [3]:
def get_pesq_mos(pesq_path, clean_wav, degraded_wav, verbose=True):
    """
    Compare degraded_wav to clean_wav using PESQ.
    Returns float MOS score in range -0.5-4.5 (?!)
    """
    
    if verbose: 
        print(f'Command: {pesq_path} +16000 {clean_wav} {degraded_wav}')
    pesq_output = !{pesq_path} +16000 {clean_wav} {degraded_wav}
    results = pesq_output[-1]
    if verbose:
        print("Raw results:", results)

    mos = float(results.split('=')[1].split('\t')[0].strip())
    if verbose:
        print("MOS:", mos)
    
    return mos

#Test PESQ
print("Test PESQ using wav:", test_file)
get_pesq_mos(pesq_path, test_file, test_file)

Test PESQ using wav: D:\Code\Denoising\E0-Mel\data\raw\17_en-US-Wavenet-A_0_14-2-C.wav
Command: D:\Code\Denoising\E0-Mel\..\Tools\pesq.exe +16000 D:\Code\Denoising\E0-Mel\data\raw\17_en-US-Wavenet-A_0_14-2-C.wav D:\Code\Denoising\E0-Mel\data\raw\17_en-US-Wavenet-A_0_14-2-C.wav
Raw results: P.862 Prediction (Raw MOS, MOS-LQO):  = 4.500	4.549
MOS: 4.5


4.5

In [4]:
def test_wav_to_mel_to_wav(test_file, output_base, output_dir, wav_to_mel_func, mel_to_wav_func,
                            sr_range, fft_range, step_range, mel_bin_range, pesq_path):

    
    res_shape = (len(sr_range), len(fft_range), len(step_range), len(mel_bin_range))
    mos_results = np.zeros(res_shape)
    mel_params = np.zeros(res_shape)
    
    total_tests = mos_results.size
    cur_test = 0
    
    for sr_idx, sr_in in enumerate(sr_range):
        orig_wav, sr = librosa.core.load(test_file, sr=sr_in)

        for fft_idx, fft_size in enumerate(fft_range):
            for step_idx, step_size in enumerate(step_range):
                for mel_idx, n_mels in enumerate(mel_bin_range):
                    
                    cur_test += 1
                    
                    output_filename = f"{output_base}_{sr}_{fft_size}_{step_size}_{n_mels}"
                    
                    M = wav_to_mel_func(orig_wav, sr, fft_size, n_mels, step_size)
                    mel_params[sr_idx, fft_idx, step_idx, mel_idx] = M.shape[0] * M.shape[1]
                    
                    mel_path = os.path.join(output_dir, f"{output_filename}.npy")
                    np.save(mel_path, M)

                    print(f"{cur_test}/{total_tests}. sr: {sr}, size: {fft_size}, step: {step_size}, n_mels: {n_mels} = {M.shape}, {np.min(M)},{np.max(M)}")

                    T = mel_to_wav_func(M, sr, fft_size, n_mels, step_size)

                    #Check the length hasn't changed
                    if len(T) != len(orig_wav):
                        print(f"Scaling output wav length ({len(T)}) to match original wav length ({len(orig_wav)})")
                        scale = len(orig_wav) / len(T)
                        new_sr = sr * scale
                        T = librosa.resample(T, sr, new_sr)
            
                    #Resample back to 16k so PESQ can work
                    resamp_T = librosa.resample(T, sr, 16000)

                    #Scale amplitude into -1 to 1 and then convert to int16
                    Tint = (resamp_T/max(resamp_T)) * (32767 * 0.9) #Don't go to max vol    
                    
                    wav_path = os.path.join(output_dir, f"{output_filename}.wav")
                    wavfile.write(wav_path,16000,Tint.astype('int16'))

                    #PESQ test against original file
                    mos = get_pesq_mos(pesq_path, test_file, wav_path, verbose=False)
                    print("Test:", sr_in, fft_size, step_size, n_mels, "=", mos)
                    mos_results[sr_idx, fft_idx, step_idx, mel_idx] = mos
                    
    return mos_results, mel_params


In [5]:
#Get best parameters from results
def get_best_parameters(results, sr_range, fft_range, step_range, mel_bin_range):
    best_indices = np.unravel_index(np.argmax(results), results.shape)
    best_mos = results[best_indices]
    print(best_indices, best_mos)
   
    return get_parameters_for_indices(best_indices, sr_range, fft_range, step_range, mel_bin_range), best_mos

def get_parameters_for_indices(indices, sr_range, fft_range, step_range, mel_bin_range):
    sr = sr_range[indices[0]]
    fft = fft_range[indices[1]]
    step = step_range[indices[2]]
    mels = mel_bin_range[indices[3]]  
    
    return (sr, fft, step, mels)

def top_n_results(results, mel_shapes, sr_range, fft_range, step_range, mel_bin_range, top_n=5):
    
    if top_n == None: #return them all
        n = None
    else:
        n = min(top_n, results.size)
        if n != top_n:
            print(f"top_n ({top_n}) > results.size ({results.size}). Returning all.")
        
    res_shape = results.shape
    unravel = np.argsort(results, axis=None)
    rev_idx = [i for i in unravel[::-1]] #reverse
    top_n_idx = rev_idx[:n]
    mos_sorted = np.unravel_index(top_n_idx, res_shape) 
    
    top_results = []
    
    for i in range(len(mos_sorted[0])):
        idx = (mos_sorted[0][i], mos_sorted[1][i], mos_sorted[2][i], mos_sorted[3][i]) 
        top_results.append( (get_parameters_for_indices(idx, sr_range, fft_range, step_range, mel_bin_range), results[idx], mel_shapes[idx]) )
        
    return top_results


## Complex MEL spectrogram without log power scaling

This is the most reversable way of doing the MEL encoding.
It maintains both real and imaginary components, and doesn't do any log scaling.

In [6]:
complex_output_dir = os.path.join(base_output_dir, "raw_mel_with_phase")
os.makedirs(complex_output_dir, exist_ok=True)

def complex_mel_to_wav(M, sr, fft_size, n_mels, step_size):
    T = avp.mel_to_time(M, sr, fft_size, n_mels, step_size)
    Tint = (T/max(T)) * (32767 * 0.9) #Don't go to max vol    
    return Tint

complex_wav_to_mel = avp.time_to_mel

complex_results, complex_mel_params = test_wav_to_mel_to_wav(test_file, output_file_base, complex_output_dir, \
                                         complex_wav_to_mel, complex_mel_to_wav, \
                                         sr_range, fft_range, step_range, mel_bin_range, pesq_path)

1/36. sr: 16000, size: 1024, step: 32, n_mels: 80 = (436, 80), (-6.703327502351268+3.5117883181990246j),(7.709026427789632+0.4048744708531423j)
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 80 = 1.141
2/36. sr: 16000, size: 1024, step: 32, n_mels: 100 = (436, 100), (-10.338551615076904-2.4593682109563804j),(11.388084649186275+0.8750742649742576j)
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 100 = 1.197
3/36. sr: 16000, size: 1024, step: 32, n_mels: 120 = (436, 120), (-15.1538277950272-7.457563429780431j),(14.992685839796026+7.5644275580739295j)
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 120 = 1.085
4/36. sr: 16000, size: 1024, step: 64, n_mels: 80 = (218, 80), (-6.703327502351268+3.5117883181990246j),(6.730106616131826-2.4028507453531174j)
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 64 80 = 1.178
5/36. sr:

Scaling output wav length (45952) to match original wav length (48000)
Test: 48000 2048 64 80 = 1.599
35/36. sr: 48000, size: 2048, step: 64, n_mels: 100 = (686, 100), (-58.501916918884554+10.392254955631007j),(54.742792366237154-11.319856713258666j)
Scaling output wav length (45952) to match original wav length (48000)
Test: 48000 2048 64 100 = 1.262
36/36. sr: 48000, size: 2048, step: 64, n_mels: 120 = (686, 120), (-93.9238966718059+14.23650577666223j),(80.38275572143961-4.136369800204549j)
Scaling output wav length (45952) to match original wav length (48000)
Test: 48000 2048 64 120 = 1.488


In [7]:
get_best_parameters(complex_results, sr_range, fft_range, step_range, mel_bin_range)

(2, 0, 0, 1) 3.003


((48000, 1024, 32, 100), 3.003)

## Test 2: Log power spectrum MEL. 

Mel spectrum is created and then log scaled (which removed the imaginary components)
Imaginary values are discarded and then re-filled with 0.0 during reconstruction.

This naive implementation returns garbage that make no sense but gives PESQ scores ~2

In [8]:
log_pow_output_dir = os.path.join(base_output_dir, "log_power")
os.makedirs(log_pow_output_dir, exist_ok=True)

def wav_to_log_pow_mel(orig_wav, sr, fft_size, n_mels, step_size):
    M = avp.time_to_mel(orig_wav, sr, fft_size, n_mels, step_size)
    mel_abs_log = np.abs(M)
    #mel_abs_log /= np.max(mel_abs_log)
    mel_abs_log = np.where(mel_abs_log == 0, np.finfo(float).eps, mel_abs_log)  # Numerical Stability
    #print("mel_abs_log minmax:", mel_abs_log.min(), mel_abs_log.max())
    mel_abs_log = np.log10(mel_abs_log) #power
    #print("mel_abs_log minmax:", mel_abs_log.min(), mel_abs_log.max())
    return mel_abs_log

def log_pow_mel_to_wav(log_pow_mel, sr, fft_size, n_mels, step_size):
    
    #Reverse log10
    pow_mel = np.power(10, log_pow_mel)
    
    reg = np.max(pow_mel) / 1E8
    
    #THIS PRODUCES GARBAGE, DO NOT USE FOR REAL.
    #Create complex numbers, fill in imag values
    #expand the complex data to 2X data with true real and image number
    expanded = np.zeros((pow_mel.shape[0],pow_mel.shape[1]*2))
    expanded[:,::2] = pow_mel
    expanded[:,1::2] = 0.0
    complex_data = avp.real_imag_collapse(expanded)
        
    #Invert mel to wav
    T = avp.mel_to_time(complex_data, sr, fft_size, n_mels, step_size)      
    return T
    
log_pow_results, log_pow_mel_params = test_wav_to_mel_to_wav(test_file, output_file_base, log_pow_output_dir, \
                                         wav_to_log_pow_mel, log_pow_mel_to_wav, \
                                         sr_range, fft_range, step_range, mel_bin_range, pesq_path)    

1/36. sr: 16000, size: 1024, step: 32, n_mels: 80 = (436, 80), -4.588235773802311,0.896627428071962
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 80 = 1.793
2/36. sr: 16000, size: 1024, step: 32, n_mels: 100 = (436, 100), -4.873463982643096,1.070218777660646
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 100 = 1.755
3/36. sr: 16000, size: 1024, step: 32, n_mels: 120 = (436, 120), -4.580125023419865,1.2302930736948794
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 32 120 = 1.778
4/36. sr: 16000, size: 1024, step: 64, n_mels: 80 = (218, 80), -4.429313442291006,0.896627428071962
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 1024 64 80 = 1.398
5/36. sr: 16000, size: 1024, step: 64, n_mels: 100 = (218, 100), -4.373750161702082,1.070218777660646
Scaling output wav length (14976) to match original wav length (16000)
Test: 16000 

In [9]:
get_best_parameters(log_pow_results, sr_range, fft_range, step_range, mel_bin_range)

(2, 0, 1, 0) 2.186


((48000, 1024, 64, 80), 2.186)

## Test 3a: Use non-iterated Griffin-Lim to estimate missing phase information.
This is the proper implementation of the naive version above.

In [22]:
gl_output_dir = os.path.join(base_output_dir, "griffin_lim")
os.makedirs(gl_output_dir, exist_ok=True)

def wmc_wav_to_log_pow_mel(orig_wav, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=0,
                    target_sample_rate=16000)
    
    conv.load_wav_data(orig_wav, sr)
    return conv.mel_spectrogram
    
def wmc_log_pow_mel_to_wav(log_pow_mel, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=0,
                    target_sample_rate=16000)
    
    conv.load_mel_data(log_pow_mel, sr)
    T = conv.wav 
    return T

gl_results, gl_mel_params = test_wav_to_mel_to_wav(test_file, output_file_base, gl_output_dir, \
                                    wmc_wav_to_log_pow_mel, wmc_log_pow_mel_to_wav, \
                                    sr_range, fft_range, step_range, mel_bin_range, pesq_path) 

1/36. sr: 16000, size: 1024, step: 32, n_mels: 80 = (80, 478), -4.0,-0.2564985752105713
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 80 = 2.071
2/36. sr: 16000, size: 1024, step: 32, n_mels: 100 = (100, 478), -4.0,-0.1744890660047531
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 100 = 1.838
3/36. sr: 16000, size: 1024, step: 32, n_mels: 120 = (120, 478), -4.0,-0.06006140261888504
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 120 = 2.096
4/36. sr: 16000, size: 1024, step: 64, n_mels: 80 = (80, 238), -4.0,-0.25689488649368286
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 80 = 2.068
5/36. sr: 16000, size: 1024, step: 64, n_mels: 100 = (100, 238), -4.0,-0.17371097207069397
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 100 = 1.867
6/36. sr: 16000, size: 1024, step: 64,

In [11]:
get_best_parameters(gl_results, sr_range, fft_range, step_range, mel_bin_range)

(1, 1, 0, 2) 2.179


((32000, 2048, 32, 120), 2.179)

In [12]:
top_gl = top_n_results(gl_results, gl_mel_params, sr_range, fft_range, step_range, mel_bin_range, top_n=10)
for r in top_gl: print(r)

((32000, 2048, 32, 120), 2.179, 114960.0)
((16000, 1024, 64, 120), 2.137, 28560.0)
((32000, 2048, 32, 80), 2.116, 76640.0)
((16000, 1024, 32, 120), 2.096, 57360.0)
((16000, 1024, 32, 80), 2.071, 38240.0)
((48000, 2048, 32, 120), 2.07, 176400.0)
((16000, 1024, 64, 80), 2.068, 19040.0)
((32000, 2048, 64, 80), 2.044, 38240.0)
((48000, 2048, 64, 80), 2.023, 58720.0)
((48000, 2048, 64, 100), 1.981, 73400.0)


## Test 3b: Use iterated Griffin-Lim, with 10 iterations, to estimate phase information.

Iterated Griffin-Lim uses multiple passes to refine the phase estimate. 

In [13]:
gl10_output_dir = os.path.join(base_output_dir, "griffin_lim_10")
os.makedirs(gl10_output_dir, exist_ok=True)

n_iterations = 10

def wmc_i_wav_to_log_pow_mel(orig_wav, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=n_iterations,
                    target_sample_rate=16000)
    
    conv.load_wav_data(orig_wav, sr)
    return conv.mel_spectrogram
    
def wmc_i_log_pow_mel_to_wav(log_pow_mel, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=n_iterations,
                    target_sample_rate=16000)
    
    conv.load_mel_data(log_pow_mel, sr)
    T = conv.wav   
    return T

gl10_results, gl10_mel_params = test_wav_to_mel_to_wav(test_file,output_file_base, gl10_output_dir, \
                                      wmc_i_wav_to_log_pow_mel, wmc_i_log_pow_mel_to_wav, \
                                      sr_range, fft_range, step_range, mel_bin_range, pesq_path) 

1/36. sr: 16000, size: 1024, step: 32, n_mels: 80 = (80, 478), -4.0,-0.2564985752105713
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 80 = 2.618
2/36. sr: 16000, size: 1024, step: 32, n_mels: 100 = (100, 478), -4.0,-0.1744890660047531
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 100 = 2.753
3/36. sr: 16000, size: 1024, step: 32, n_mels: 120 = (120, 478), -4.0,-0.06006140261888504
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 120 = 2.779
4/36. sr: 16000, size: 1024, step: 64, n_mels: 80 = (80, 238), -4.0,-0.25689488649368286
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 80 = 2.912
5/36. sr: 16000, size: 1024, step: 64, n_mels: 100 = (100, 238), -4.0,-0.17371097207069397
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 100 = 2.903
6/36. sr: 16000, size: 1024, step: 64,

In [14]:
get_best_parameters(gl10_results, sr_range, fft_range, step_range, mel_bin_range)

(2, 1, 1, 1) 2.978


((48000, 2048, 64, 100), 2.978)

In [15]:
top_gl10 = top_n_results(gl10_results, gl10_mel_params, sr_range, fft_range, step_range, mel_bin_range, top_n=10)
for r in top_gl10: print(r)

((48000, 2048, 64, 100), 2.978, 73400.0)
((32000, 2048, 64, 120), 2.945, 57360.0)
((32000, 2048, 64, 80), 2.935, 38240.0)
((16000, 1024, 64, 80), 2.912, 19040.0)
((16000, 1024, 64, 120), 2.907, 28560.0)
((16000, 1024, 64, 100), 2.903, 23800.0)
((48000, 2048, 32, 80), 2.868, 117600.0)
((32000, 2048, 32, 80), 2.859, 76640.0)
((48000, 2048, 32, 100), 2.856, 147000.0)
((48000, 2048, 64, 80), 2.843, 58720.0)


## Test 3c: 20 iteration Griffin-Lim

In [16]:
gl20_output_dir = os.path.join(base_output_dir, "griffin_lim_20")
os.makedirs(gl20_output_dir, exist_ok=True)

n_iterations = 20

def wmc_i20_wav_to_log_pow_mel(orig_wav, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=n_iterations,
                    target_sample_rate=16000)
    
    conv.load_wav_data(orig_wav, sr)
    return conv.mel_spectrogram
    
def wmc_i20_log_pow_mel_to_wav(log_pow_mel, sr, fft_size, n_mels, step_size):
    
    conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                    mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                    shorten_factor=1, griffin_lim_iterations=n_iterations,
                    target_sample_rate=16000)
    
    conv.load_mel_data(log_pow_mel, sr)
    T = conv.wav
    return T

gl20_results, gl20_mel_params = test_wav_to_mel_to_wav(test_file, output_file_base, gl20_output_dir, \
                                      wmc_i20_wav_to_log_pow_mel, wmc_i20_log_pow_mel_to_wav, \
                                      sr_range, fft_range, step_range, mel_bin_range, pesq_path) 

1/36. sr: 16000, size: 1024, step: 32, n_mels: 80 = (80, 478), -4.0,-0.2564985752105713
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 80 = 2.737
2/36. sr: 16000, size: 1024, step: 32, n_mels: 100 = (100, 478), -4.0,-0.1744890660047531
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 100 = 2.758
3/36. sr: 16000, size: 1024, step: 32, n_mels: 120 = (120, 478), -4.0,-0.06006140261888504
Scaling output wav length (15808) to match original wav length (16000)
Test: 16000 1024 32 120 = 2.826
4/36. sr: 16000, size: 1024, step: 64, n_mels: 80 = (80, 238), -4.0,-0.25689488649368286
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 80 = 2.441
5/36. sr: 16000, size: 1024, step: 64, n_mels: 100 = (100, 238), -4.0,-0.17371097207069397
Scaling output wav length (15744) to match original wav length (16000)
Test: 16000 1024 64 100 = 2.222
6/36. sr: 16000, size: 1024, step: 64,

In [17]:
get_best_parameters(gl20_results, sr_range, fft_range, step_range, mel_bin_range)

(2, 1, 1, 1) 2.932


((48000, 2048, 64, 100), 2.932)

In [18]:
top_gl20 = top_n_results(gl20_results, gl20_mel_params, sr_range, fft_range, step_range, mel_bin_range, top_n=10)
for r in top_gl20: print(r)

((48000, 2048, 64, 100), 2.932, 73400.0)
((48000, 2048, 64, 80), 2.913, 58720.0)
((32000, 2048, 64, 120), 2.909, 57360.0)
((48000, 2048, 64, 120), 2.902, 88080.0)
((32000, 2048, 32, 80), 2.869, 76640.0)
((32000, 2048, 32, 120), 2.837, 114960.0)
((16000, 1024, 32, 120), 2.826, 57360.0)
((32000, 2048, 64, 100), 2.814, 47800.0)
((48000, 2048, 32, 120), 2.802, 176400.0)
((48000, 2048, 32, 100), 2.8, 147000.0)


## Results analysis

What parameters should we choose to do MEL encoding?

In [19]:
for i in range(10):
    print(top_gl[i], top_gl10[i], top_gl20[i])

((32000, 2048, 32, 120), 2.179, 114960.0) ((48000, 2048, 64, 100), 2.978, 73400.0) ((48000, 2048, 64, 100), 2.932, 73400.0)
((16000, 1024, 64, 120), 2.137, 28560.0) ((32000, 2048, 64, 120), 2.945, 57360.0) ((48000, 2048, 64, 80), 2.913, 58720.0)
((32000, 2048, 32, 80), 2.116, 76640.0) ((32000, 2048, 64, 80), 2.935, 38240.0) ((32000, 2048, 64, 120), 2.909, 57360.0)
((16000, 1024, 32, 120), 2.096, 57360.0) ((16000, 1024, 64, 80), 2.912, 19040.0) ((48000, 2048, 64, 120), 2.902, 88080.0)
((16000, 1024, 32, 80), 2.071, 38240.0) ((16000, 1024, 64, 120), 2.907, 28560.0) ((32000, 2048, 32, 80), 2.869, 76640.0)
((48000, 2048, 32, 120), 2.07, 176400.0) ((16000, 1024, 64, 100), 2.903, 23800.0) ((32000, 2048, 32, 120), 2.837, 114960.0)
((16000, 1024, 64, 80), 2.068, 19040.0) ((48000, 2048, 32, 80), 2.868, 117600.0) ((16000, 1024, 32, 120), 2.826, 57360.0)
((32000, 2048, 64, 80), 2.044, 38240.0) ((32000, 2048, 32, 80), 2.859, 76640.0) ((32000, 2048, 64, 100), 2.814, 47800.0)
((48000, 2048, 64, 80),

10 iterations of Griffin-Lim are worth about 1 point on the PESQ scale.

An improvement of 2 to 3 is huge and you can tell when you listen to the output.

(48000, 2048, 64, 100) is top of both 10 and 20 iteration GL and has 73400 parameters ie (100, 734) for a 1 second of audio.

## Test 'shortening'
The MEL implementation we are using has the option to 'shorten' the produced 2D array by zooming it.

This reduces the number of parameters in the MEL output but at the loss of some information.

The following tests try different shortening factors on the basic parameters of (48000, 2048, 64, 100)

In [20]:
gl10s_output_dir = os.path.join(base_output_dir, "griffin_lim_10_short")
os.makedirs(gl10s_output_dir, exist_ok=True)

n_iterations = 10
sr_param = [48000]
fft_param = [2048]
step_param = [64]
n_mel_param = [100]

shorten_factors = [1,2,4,8,10]
shorten_results = []

for factor in shorten_factors:

    shorten_output_file_base = f"{output_file_base}_{factor}"
    
    def wmc_s_wav_to_log_pow_mel(orig_wav, sr, fft_size, n_mels, step_size):

        conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                        mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                        shorten_factor=factor, griffin_lim_iterations=n_iterations,
                        target_sample_rate=16000)

        conv.load_wav_data(orig_wav, sr)
        return conv.mel_spectrogram

    def wmc_s_log_pow_mel_to_wav(log_pow_mel, sr, fft_size, n_mels, step_size):

        conv = wmc.WavMelConverter(fft_size=fft_size, fft_step_size=step_size, spec_thresh=4, 
                        mel_n_freq_comps=n_mels, mel_start_freq=300, mel_end_freq=8000,
                        shorten_factor=factor, griffin_lim_iterations=n_iterations,
                        target_sample_rate=16000)

        conv.load_mel_data(log_pow_mel, sr)
        T = conv.wav    
        return T

    gl10s_results, gl10s_shapes = test_wav_to_mel_to_wav(test_file,shorten_output_file_base, gl10s_output_dir, \
                                          wmc_s_wav_to_log_pow_mel, wmc_s_log_pow_mel_to_wav, \
                                          sr_param, fft_param, step_param, n_mel_param, pesq_path) 
    
    shorten_results.append((gl10s_results, gl10s_shapes))

1/1. sr: 48000, size: 2048, step: 64, n_mels: 100 = (100, 734), -4.0,-7.708015540945634e-13
Test: 48000 2048 64 100 = 2.978
1/1. sr: 48000, size: 2048, step: 64, n_mels: 100 = (100, 366), -4.086615085601807,-0.00026292112306691706
Scaling output wav length (47872) to match original wav length (48000)
Test: 48000 2048 64 100 = 2.983
1/1. sr: 48000, size: 2048, step: 64, n_mels: 100 = (100, 182), -4.090807914733887,-0.0029260225128382444
Scaling output wav length (47616) to match original wav length (48000)
Test: 48000 2048 64 100 = 3.113
1/1. sr: 48000, size: 2048, step: 64, n_mels: 100 = (100, 90), -4.083613872528076,-0.014429043047130108
Scaling output wav length (47104) to match original wav length (48000)
Test: 48000 2048 64 100 = 3.107


c:\users\benhe\.conda\envs\homl2\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


1/1. sr: 48000, size: 2048, step: 64, n_mels: 100 = (100, 72), -4.081106662750244,-0.0008944268338382244
Scaling output wav length (47104) to match original wav length (48000)
Test: 48000 2048 64 100 = 3.204


In [21]:
print("Shorten factor, MOS, no. params")
for i, res in enumerate(shorten_results):
    print(shorten_factors[i], res[0].flat[0], int(res[1].flat[0]))

Shorten factor, MOS, no. params
1 2.978 73400
2 2.983 36600
4 3.113 18200
8 3.107 9000
10 3.204 7200


Compressing the 2D MEL params by up to a factor 10 bumps up the MOS, and perceptually they sound the same.

The large reduction in parameters should make the model quicker to train

# Conclusion

- Upscaling the wav file to 48kHz improves results when converting back from MEL to wav, even when the output is resampled back to 16kHz.
- 10 iterations of the Griffin-Lim algorithm improves PESQ score by ~1 point, from 2.2 to 3.0, and sounds mcuh better.
- 'Shortening' the resulting MEL spectrogram reduces the number of variables without affecting PESQ score. Greater than a factor of 10 there are internal errors with mis-sized arrays that are non-trivial to fix.


Parameter seaching shows the best parameters are: 
- Sample rate: 48000
- FFT size: 2048
- Windows step: 64
- Number of MEL bins: 100
- GL iterations: 10
- Shortening factor: 10